In [ ]:
import sqlite3
import os.path

In [ ]:
def create_or_open_db(filename):
    file_exists = os.path.isfile(filename)
    conn = sqlite3.connect(filename)
    if file_exists:
        print(''' "{}" database successfully opened '''.format(filename))
    else:
        print(''' "{}" database successfully created '''.format(filename))
    return conn

In [ ]:
conn = create_or_open_db('mycds.sqlite')
cur = conn.cursor()
cur.execute('SELECT SQLITE_VERSION()')
print('version:', cur.fetchone()) 

In [ ]:
def create_tbl_artists(conn):
    sql = '''create table if not exists Artists(
            ArtistID INTEGER PRIMARY KEY,
            ArtistName TEXT);'''
    conn.execute(sql) # shortcut for conn.cursor().execute(sql)
    print("Created Artists table successfully")

create_tbl_artists(conn)

In [ ]:
conn.execute("insert into Artists (ArtistID, ArtistName) values (NULL,'Peter Gabriel');")
conn.execute("insert into Artists (ArtistName) values ('Bruce Hornsby');")

Je kunt handig gebruik maken van de Python template notatie (*hier nog uit te werken*):

In [ ]:
conn.execute("insert into Artists (ArtistID, ArtistName) values (?, ?);",(3, 'Lyle Lovett'))

In [ ]:
names = [{'name':'Beach Boys'},{'name':'Santana'}]
conn.executemany("insert into Artists (ArtistName) values (:name);", names)

Save changes:

In [ ]:
conn.commit()

In [ ]:
def print_tbl(conn, tablename):
    #cur = conn.cursor()
    #cur.execute("select * from Artists;")
    #for row in cur:
    for row in conn.execute("select * from {0};".format(tablename)):
        print(row)

In [ ]:
print_tbl(conn, "Artists")

In [ ]:
import pandas as pd

In [ ]:
def create_tbl_CDs(conn):
    sql = '''create table if not exists CDs(
            CDID INTEGER PRIMARY KEY AUTOINCREMENT,
            ArtistID INTEGER NOT NULL,
            Title TEXT NOT NULL,
            Date TEXT);'''
    conn.execute(sql) # shortcut for conn.cursor().execute(sql)
    print("Created CDs table successfully")

In [ ]:
create_tbl_CDs(conn)

In [ ]:
cd = (1,'So','1984')
sql = "insert into CDs (ArtistID,Title,Date) values (?,?,?);"
cur = conn.cursor()
cur.execute(sql, cd)
print(cur.lastrowid)

In [ ]:
lst = [(1,'Us','1992'),
       (2,'The Way It Is','1986'),
       (2,'Scenes from the Southside','1990')]
cur.executemany("insert into CDs (ArtistID,Title,Date) values (?,?,?);", lst)
print(cur.lastrowid)

In [ ]:
sql = "insert into CDs (ArtistID,Title,Date) values (:id,:title,:date);"
dict = [{'id':1,'title':'Security','date':'1990'},
        {'id':3,'title':'Joshua Judges Ruth','date':'1992'},
        {'id':4,'title':'Pet Sounds','date':'1966'}]
cur.executemany(sql, dict)
conn.commit()

In [ ]:
# save into the DB and print
conn.commit()
print_tbl(conn, "CDs")

In [ ]:
from sqlalchemy import create_engine # database connection

In [ ]:
disk_engine = create_engine('sqlite:///mycds.sqlite')

In [ ]:
df = pd.read_sql_query('SELECT * FROM Artists LIMIT 3', disk_engine)
df.head()

In [ ]:
df = pd.read_sql_query('SELECT * FROM CDs', disk_engine)
df.head(10)

In [ ]:
df = pd.read_sql_query('SELECT * FROM CDs WHERE Date=1990', disk_engine)
df.head(10)

In [ ]:
cur = conn.cursor()    
cur.execute("SELECT * FROM CDs")

rows = cur.fetchall()

for row in rows:
        print(row)

In [ ]:
df = pd.DataFrame(rows, columns=["CDID", "ArtistID", "title", "date"])

In [ ]:
df.head()

In [ ]:
rows

In [ ]:
from IPython.display import display, HTML

# Assuming that dataframes df1 and df2 are already defined:
print("Dataframe a:")
display(df)
print("Dataframe b:")
HTML(df.to_html())
display(rows)

In [ ]:
!sqlite3 -help

In [ ]:
%%bash
sqlite3 mycds.sqlite
.schema Artists
SELECT * FROM Artists;
.tables
.schema CDs
SELECT * FROM CDs;
.quit

In [ ]:
sql = "SELECT name, sql FROM sqlite_master WHERE type='table' ORDER BY name;"
cur.execute(sql)
tables = cur.fetchall()

display(tables)

In [ ]:
sql = "PRAGMA table_info(Artists);"
cur.execute(sql)
headers = cur.fetchall()

display(headers)